In [27]:
from IPython.utils import io
import csv
from sklearn.model_selection import cross_val_predict, cross_val_score
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/BowTF/BowTF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
   #%run ../../2-FeatureExtraction/TF-IDF/TF-IDF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
   #%run ../../2-FeatureExtraction/ContextualEmbeddings/ContextualEmbeddings.ipynb import x, X, y1, y2, Y1, Y2, Features, Preprocesss
   #%run ../../2-FeatureExtraction/BagOfWords/BagOfWords.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
if Features["method"] ==  "contextual":
   x, X = x[:,0,:], X[:,0,:]


In [28]:
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

#### SVM Training

In [29]:
kernel = 'poly'
C = 3.0
WeightedLoss = False
Oversampling = True
CrossValidation = False

model = svm.SVC(kernel=kernel,C=C, class_weight='balanced' if WeightedLoss else None )

ModelInfo = {
   "Model": "SVM-1",
   "Kernel":kernel,
   "Weighted Loss": WeightedLoss,
   "Oversampling": Oversampling,
   "CrossValidation": CrossValidation,
   "C": C
}

if Oversampling:
   sm = SMOTE(random_state=42)
   x, y1 = sm.fit_resample(x, y1)

#### SVM Prediction

In [30]:
if CrossValidation:
   X_c, Y_c1 = np.concatenate([x.A, X.A]), np.concatenate([y1, Y1])
   outputs = cross_val_predict(model, X_c, Y_c1, cv=5)
   Report = classification_report(Y_c1, outputs, output_dict=True)
   print(classification_report(Y_c1, outputs))
   F1 = f1_score(Y_c1, outputs, average='macro')
   print('F1 score: ', F1)

else:
   model = model.fit(x, y1)
   outputs = model.predict(X)
   Report = classification_report(Y1, outputs, output_dict=True)
   print(classification_report(Y1, outputs))
   F1 = f1_score(Y1, outputs, average='macro')
   print('F1 score: ', F1)

#### Log Model Info

In [ ]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)